In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
path_to_data = 'data/ECU_IoHT.xlsx'
data = pd.read_excel(path_to_data)

In [3]:
def extract_source_destination_port(text):
    match = re.search(r'(\d+)\s*>\s*(\d+)', text)
    if match:
        source, destination = match.groups()
        # Remove the matched substring from text
        updated_text = text.replace(match.group(0), '', 1).strip()
        return int(source), int(destination), updated_text
    else:
        return np.nan, np.nan, text

In [4]:
df = pd.DataFrame()
df[['source_p', 'destination_p', 'updated_info']] = data.Info.apply(lambda x: pd.Series(extract_source_destination_port(x)))

In [5]:
df['updated_info'].nunique(), data['Info'].nunique()

(8833, 24010)

In [6]:
def extract_seq(text):
    # Adjust regex to match the sequence number
    match = re.search(r'seq\s*=\s*(\d+)', text, re.IGNORECASE)  #
    if match:
        Seq = match.group(1)  # Get the first group directly
        updated_text = text.replace(match.group(0), '', 1).strip()
        return int(Seq), updated_text
    else:
        return np.nan, text


In [7]:
df2 = pd.DataFrame()
df2[['seq', 'updated_info']] = df['updated_info'].apply(lambda x: pd.Series(extract_seq(x)))

In [8]:
df2['updated_info'].nunique(), df['updated_info'].nunique(), data['Info'].nunique()

(7677, 8833, 24010)

In [9]:
df2[df2['seq'].notna()]['seq'].nunique()

2379

In [10]:
def extract_transaction_id(text):
    # Regex to match transaction ID starting with 0x and followed by hexadecimal characters
    match = re.search(r'0x[0-9a-fA-F]+', text)
    
    if match:
        # Extract the transaction ID as a string
        transaction_id = match.group(0)
        
        # Remove the transaction ID from the original text
        updated_text = text.replace(transaction_id, '').strip()  # Strip to clean up any leading/trailing spaces
        
        # Return the integer value of the transaction ID and the updated text
        return int(transaction_id, 16), updated_text  # Convert hex to integer and return
        
    else:
        # If no transaction ID is found, return np.nan for both values
        return np.nan, text


In [11]:
df3 = pd.DataFrame()
df3[['transaction_id', 'updated_info']] = df2['updated_info'].apply(lambda x: pd.Series(extract_transaction_id(x)))

In [12]:
df3['updated_info'].nunique(), df2['updated_info'].nunique(), df['updated_info'].nunique(), data['Info'].nunique()

(7166, 7677, 8833, 24010)

In [ ]:
def extract_len(text):
    match = re.search(r'len=(\d+)', text.lower())
    if match:
        length = match.group(1) 
        updated_text = text.replace(length, '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan  

In [4]:
def extract_source_destination_port(text):
    match = re.search(r'(\d+)\s*>\s*(\d+)', text)
    if match:
        source, destination = match.groups()
        return int(source), int(destination)
    else:
        return np.nan
    


def extract_source_destination_port(text):
    match = re.search(r'(\d+)\s*>\s*(\d+)', text)
    if match:
        source, destination = match.groups()
        # Remove the matched substring from text
        updated_text = text.replace(match.group(0), '', 1).strip()
        return int(source), int(destination), updated_text
    else:
        return np.nan, np.nan, text


In [5]:
path_to_data = 'data/ECU_IoHT.xlsx'
data = pd.read_excel(path_to_data)

In [7]:
df = pd.DataFrame()
df[['source_p', 'destination_p']] = data.Info.apply(lambda x: pd.Series(extract_source_destination_port(x)))

In [8]:
df[df['source_p'].notna() & df['destination_p'].notna()]

,source_p,destination_p
6,36954.0,1720.0
7,36954.0,5900.0
8,36954.0,135.0
9,36954.0,143.0
10,36954.0,993.0
...,...,...
111197,3850.0,443.0
111198,3850.0,443.0
111199,3850.0,443.0
111200,3850.0,443.0


In [9]:
df[['len']] = data.Info.apply(lambda x: pd.Series(extract_len(x)))

In [10]:
df[df['len'].notna()]

,source_p,destination_p,len
6,36954.0,1720.0,0.0
7,36954.0,5900.0,0.0
8,36954.0,135.0,0.0
9,36954.0,143.0,0.0
10,36954.0,993.0,0.0
...,...,...,...
111197,3850.0,443.0,69.0
111198,3850.0,443.0,0.0
111199,3850.0,443.0,341.0
111200,3850.0,443.0,0.0


In [11]:
df['Transaction_ID'] = data['Info'].apply(extract_transaction_id)

In [17]:
df[df['Transaction_ID'].notna()]['Transaction_ID']

3         0x0c44
5         0x0c44
2805      0x5459
2806      0x5459
6086      0xc082
           ...  
111121    0x0021
111153    0x00df
111154    0x00df
111185    0x0021
111193    0x00df
Name: Transaction_ID, Length: 78994, dtype: object

In [26]:
pd.DataFrame(data.Info.unique()).to_csv('info.csv')